<a href="https://colab.research.google.com/github/PANDASANG1231/deeplearn_note/blob/main/029_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
sys.path.append('/content/drive/MyDrive/Colab Notebooks/deeplearning_note')
from tool import *

Mounted at /content/drive


In [ ]:
import torch
import torchvision

In [ ]:
class Resnet_block(torch.nn.Module):

  def __init__(self, in_channels, out_channels, if_double, if_conv):

    super().__init__()
    self.if_double = if_double
    self.if_conv = if_conv
    self.conv2d_3 = None

    if if_double:
      self.conv2d_1 = torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels, stride=2, kernel_size=3, padding=1)
      self.conv2d_2 = torch.nn.Conv2d(in_channels=out_channels, out_channels=out_channels, stride=1, kernel_size=3, padding=1)

      if if_conv:
        self.conv2d_3 = torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels, stride=2, kernel_size=1, padding=0)

    else:
      self.conv2d_1 = torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels, stride=1, kernel_size=3, padding=1)
      self.conv2d_2 = torch.nn.Conv2d(in_channels=out_channels, out_channels=out_channels, stride=1, kernel_size=3, padding=1)

      if if_conv:
        self.conv2d_3 = torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels, stride=1, kernel_size=1, padding=0)

    self.bn1 = torch.nn.BatchNorm2d(num_features=out_channels)
    self.bn2 = torch.nn.BatchNorm2d(num_features=out_channels)
    self.relu = torch.nn.ReLU()

  def forward(self, X):

    Y = self.relu(self.bn1(self.conv2d_1(X)))
    Y = self.bn2(self.conv2d_2(Y))

    if self.conv2d_3:
      X = self.conv2d_3(X)
    
    Y += X
    return self.relu(Y)


In [ ]:
X = torch.ones((1,1,28, 28))

for layer in ResNet:
  X = layer(X)
  print(layer.__class__.__name__, X.shape)

Sequential torch.Size([1, 64, 14, 14])
Sequential torch.Size([1, 128, 7, 7])
Sequential torch.Size([1, 256, 4, 4])
Sequential torch.Size([1, 512, 2, 2])
AvgPool2d torch.Size([1, 512, 1, 1])
Flatten torch.Size([1, 512])
Linear torch.Size([1, 10])


Training with Resnet

In [ ]:
train_dataset = torchvision.datasets.FashionMNIST(root="./",train=True,download=True,transform=torchvision.transforms.ToTensor())
test_dataset = torchvision.datasets.FashionMNIST(root="./",train=False,download=True,transform=torchvision.transforms.ToTensor())

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/raw



In [ ]:
batch_size = 256

ResNet = torch.nn.Sequential(
    *[
     torch.nn.Sequential(*[Resnet_block(1, 64, True, True), Resnet_block(64, 64, False, True), 
                           Resnet_block(64, 64, False, True)]), 
     torch.nn.Sequential(*[Resnet_block(64, 128, True, True), Resnet_block(128, 128, False, True), 
                           Resnet_block(128, 128, False, True)]), 
     torch.nn.Sequential(*[Resnet_block(128, 256, True, True), Resnet_block(256, 256, False, True), 
                           Resnet_block(256, 256, False, True)]), 
     torch.nn.Sequential(*[Resnet_block(256, 512, True, True), Resnet_block(512, 512, False, True), 
                           Resnet_block(512, 512, False, True)]), 
     torch.nn.AvgPool2d(kernel_size=(2, 2)),
     torch.nn.Flatten(),
     torch.nn.Linear(512, 10),
    ]
)


train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               shuffle=True)

test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                              batch_size=batch_size,
                                              shuffle=False)

device = torch.device('cuda')
loss = torch.nn.CrossEntropyLoss()


train_p2(epoch_num=10, 
         model=ResNet,  
         loss=loss, 
         lr=0.5, 
         train_data_iter=train_dataloader,
         test_data_iter=test_dataloader,
         device=device,
         optim_type="ADAM")


loss 1.068, train acc 0.669, test acc 0.795
loss 2.407, train acc 0.125, test acc 0.177
loss 0.909, train acc 0.642, test acc 0.795
loss 0.475, train acc 0.823, test acc 0.834
loss 0.394, train acc 0.853, test acc 0.845
loss 0.367, train acc 0.863, test acc 0.860
loss 0.339, train acc 0.873, test acc 0.864
loss 0.332, train acc 0.875, test acc 0.871
loss 0.318, train acc 0.880, test acc 0.870
loss 0.306, train acc 0.885, test acc 0.882
Calculation Ability: 3500.2 examples/sec on cuda
